In [1]:
import sys
from sklearn import svm
import adversaries as ad
from data_reader import input, output, operations
import learners as learner
from metrics import EvasionMetrics
from copy import deepcopy
from data_reader.dataset import EmailDataset

# Library Demo

Here we demonstrate framework testing by simulating an SVM classifier and its  
resulting classification accuracy after an attack from the simple_optimize adversary.

## Initial Learner Training
- choose an initial learner model,
- load the instances for training,
- and wrap the two with the Classifier wrapper. 


In [2]:
from learners import SimpleLearner
learning_model = svm.SVC(probability=True, kernel='linear')
dataset = EmailDataset(path='./data_reader/data/test/100_instance_debug.csv', raw=False)
trainset, testset = dataset.split({'train': 60, 'test': 40})

clf = SimpleLearner(model=learning_model, training_instances=trainset)

# instances = input.load_instances('./data_reader/data/test/100_instance_debug')

# clf = Classifier(learning_model, instances)


<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>


In [3]:
x = [y for y in dataset]

In [4]:
import numpy as np
a = np.array([0,1,2])

In [5]:
a[1]=5

In [6]:
a

array([0, 5, 2])

In [7]:
trainset.features.todense()[0].shape


(1, 1000)

In [8]:
trainset

In [9]:
y = [x for x in trainset[0]]

In [10]:
X, y = trainset.numpy()

In [11]:
X

array([[ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
clf.train() # train the initial learner

In [13]:
len(trainset)

60

## Adversary Attack 
Now we choose an adversary attack strategy to transform the 
original instances, here we choose simple_optimize.

In [14]:
print(trainset.numpy().features.shape)

(60, 1000)


In [15]:
# initialize attacker from the ad package
adv = ad.SimpleOptimize(lambda_val=-100, max_change = 65, learner = clf)
# perform the attack on the data to transform instances
instances_post_attack = adv.attack(trainset)

/Users/amdegroot/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [16]:
type(instances_post_attack)

data_reader.dataset.EmailDataset

In [17]:
dataset.features[0,3]

1.0

In [18]:
type(instances_post_attack.features)

scipy.sparse.csr.csr_matrix

### Graph results of pre attack vs. post attack learner 

In [19]:
instances_post_attack

In [20]:
# obtain metrics of classification performance both pre- and post- attack
metrics = EvasionMetrics(clf, testset, instances_post_attack)
preAccuracy = metrics.getAccuracy(True)
postAccuracy = metrics.getAccuracy(False)
print ("Pre-attack Accuracy (naive classifier): " + str(preAccuracy))
print ("Post-attack Accuracy: (naive classifier): " + str(postAccuracy))

# plot ROC curve of naive classifier performance after attack
metrics.plotROC(False)

Pre-attack Accuracy (naive classifier): 1.0
Post-attack Accuracy: (naive classifier): 1.0


NotImplementedError: 

## Learner Response
Here we wrap a defense strategy for the learner to exercise
during the next round of training to become more robust. 

In [21]:
dataset2 = EmailDataset(path='./data_reader/data/test/100_instance_debug.csv', raw=False) # load instances
trainset2, testset2 = dataset2.split({'train': 60, 'test': 40})
learning_model2 = svm.SVC(probability=True, kernel='linear')
clf2 = learner.Retraining(learning_model2,trainset2, {'attack_alg': ad.SimpleOptimize,'adv_params':{}})
clf2.train() # train again with defense strategy 

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>
==> Training...
EmailDataset(features=<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 102 stored elements in Compressed Sparse Row format>, labels=1.0)
EmailDataset(features=<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 139 stored elements in Compressed Sparse Row format>, labels=1.0)
EmailDataset(features=<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 106 stored elements in Compressed Sparse Row format>, labels=1.0)
EmailDataset(features=<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 139 stored elements in Compressed Sparse Row format>, labels=1.0)
EmailDataset(features=<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 139 stored elements in Compressed Sparse Row format>, labels=1.0)
EmailDataset(features=<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 117 stored elements in Compressed Sparse Row format>, labels=1.0)
EmailDataset(features=

## Adversary Attack

In [23]:
# Now we launch the attack on the robust classifier 
adv = ad.SimpleOptimize(lambda_val=-100, max_change = 65, learner = clf2)
instances_post_attack2 = adv.attack(trainset2)


NotImplementedError: 

### Display metrics after attack on the robust classifier
Here we see the improvements made in accuracy as a result 
of the chosen defense strategy.  Clearly, our learner has 
become more robust to adversary attacks.

In [ ]:
# obtain metrics of classification performance both pre- and post- attack
metrics2 = EvasionMetrics(clf2, instances, instances_post_attack2)
preAccuracy2 = metrics2.getAccuracy(True)
postAccuracy2 = metrics2.getAccuracy(False)
print ("Pre-attack Accuracy (robust classifier): " + str(preAccuracy2))
print ("Post-attack Accuracy: (robust classifier): " + str(postAccuracy2))

metrics2.plotROC(True)